## Create Tables for Metric Data on Trino 

In this notebook, we programatically create sql tables on Trino from metric parquet files generated by analyzing Testgrid and stored on S3. These metric files correspond to various [key performance indicator metrics](metrics/README.md) that are relevant to various personas (developer, manager, etc.) involved in the CI process.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import trino

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,
s3_endpoint_url = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")
AUTOMATION = os.getenv("IN_AUTOMATION")

In [4]:
# Create an Trino client
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(
        os.environ["TRINO_USER"],
        os.environ["TRINO_PASSWD"]
    ),
    host=os.environ["TRINO_HOST"],
    port=int(os.environ["TRINO_PORT"]),
    http_scheme="https",
    verify=True,
)
cur = conn.cursor()

In [5]:
test_pass_failures = """CREATE TABLE IF NOT EXISTS hive.default.test_pass_failures (
          timestamp timestamp,
          tab varchar,
          grid varchar,
          test varchar,
          failure boolean,
          passing boolean
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/test_pass_failures',
   format = 'PARQUET'
)"""


In [6]:
cur.execute(test_pass_failures)
cur.fetchall()

[[True]]

In [7]:
## Check if the table is there
cur.execute("select * from hive.default.test_pass_failures LIMIT 2")
cur.fetchall()

[['2021-09-20 00:00:47.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  True,
  False],
 ['2021-09-18 07:45:05.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  True,
  False]]

In [8]:
blocked_timed_out = """CREATE TABLE IF NOT EXISTS hive.default.blocked_timed_out (
        timestamp timestamp,
        tab varchar,
        grid varchar,
        test_blocked boolean,
        test_timed_out boolean
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/blocked_timed_out',
   format = 'PARQUET'
)"""

In [9]:
cur.execute(blocked_timed_out)
cur.fetchall()

[[True]]

In [10]:
## Check if the table is there
cur.execute("select * from hive.default.blocked_timed_out LIMIT 2")
cur.fetchall()

[['2021-09-20 00:00:47.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  False,
  False],
 ['2021-09-18 07:45:05.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  False,
  False]]

In [11]:
number_of_flakes = """CREATE TABLE IF NOT EXISTS hive.default.number_of_flakes (
        timestamp timestamp,
        tab varchar,
        grid varchar,
        test varchar,
        flake boolean
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/number_of_flakes',
   format = 'PARQUET'
)
"""

In [12]:
cur.execute(number_of_flakes)
cur.fetchall()

[[True]]

In [13]:
## Check if the table is there
cur.execute("select * from hive.default.number_of_flakes LIMIT 2")
cur.fetchall()

[['2021-09-20 00:00:47.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  False],
 ['2021-09-18 07:45:05.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  False]]

In [14]:
avg_correlation = """CREATE TABLE IF NOT EXISTS hive.default.avg_correlation (
        timestamp timestamp,
        average_number_of_correlated_failures double
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/avg_correlation',
   format = 'PARQUET'
)"""

In [15]:
cur.execute(avg_correlation)
cur.fetchall()

[[True]]

In [16]:
## Check if the table is there
cur.execute("select * from hive.default.avg_correlation LIMIT 2")
cur.fetchall()

[['2021-09-21 14:32:20.854', 8609.649731515903]]

In [17]:
correlation = """CREATE TABLE IF NOT EXISTS hive.default.correlation (
        test_name varchar,
        correlated_tests varchar
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/correlation',
   format = 'PARQUET'
)"""

In [18]:
cur.execute(correlation)
cur.fetchall()

[[True]]

In [19]:
## Check if the table is there
cur.execute("select * from hive.default.correlation LIMIT 1")
cur.fetchall()

[['openshift-tests.[sig-storage] In-tree Volumes [Driver: emptydir] [Testpattern: Dynamic PV (default fs)] fsgroupchangepolicy (Always)[LinuxOnly], pod created with an initial fsgroup, new pod fsgroup applied to volume contents [Suite:openshift/conformance/parallel] [Suite:k8s]',
  "[('openshift-tests.[sig-storage] CSI Volumes [Driver: csi-hostpath] [Testpattern: Inline-volume (default fs)] subPath should be able to unmount after the subpath directory is deleted [LinuxOnly] [Suite:openshift/conformance/parallel] [Suite:k8s]', 1.0), ('openshift-tests.[sig-storage] In-tree Volumes [Driver: local][LocalVolumeType: dir-link] [Testpattern: Dynamic PV (default fs)] fsgroupchangepolicy (Always)[LinuxOnly], pod created with an initial fsgroup, volume contents ownership changed in first pod, new pod with different fsgroup applied to the volume contents [Suite:openshift/conformance/parallel] [Suite:k8s]', 1.0), ('openshift-tests.[sig-storage] In-tree Volumes [Driver: local][LocalVolumeType: bloc

In [20]:
pct_fixed_each_ts = """CREATE TABLE IF NOT EXISTS hive.default.pct_fixed_each_ts (
        timestamp timestamp,
        tab varchar,
        grid varchar,
        pct_fixed double
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/pct_fixed_each_ts',
   format = 'PARQUET'
)"""

In [21]:
cur.execute(pct_fixed_each_ts)
cur.fetchall()

[[True]]

In [22]:
## Check if the table is there
cur.execute("select * from hive.default.pct_fixed_each_ts LIMIT 2")
cur.fetchall()

[['2021-09-07 16:37:10.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  0.0],
 ['2021-09-07 17:59:23.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  0.0]]

In [23]:
persistent_failures = """CREATE TABLE IF NOT EXISTS hive.default.persistent_failures (
        tab varchar,
        grid varchar,
        test varchar,
        consec_fail_rate double,
        mean_fail_len double,
        mean_time_to_fix double,
        pass_to_fail_rate double,
        fail_to_pass_rate double
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/persistent_failures',
   format = 'PARQUET'
)"""

In [24]:
cur.execute(persistent_failures)
cur.fetchall()

[[True]]

In [25]:
## Check if the table is there
cur.execute("select * from hive.default.persistent_failures LIMIT 2")
cur.fetchall()

[['"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  0.5,
  3.0,
  147581500.0,
  0.125,
  0.0625],
 ['"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'operator.Run multi-stage test e2e-metal-assisted - e2e-metal-assisted-baremetalds-assisted-setup container test',
  0.0,
  1.0,
  144942000.0,
  0.0625,
  0.0]]

In [26]:
build_pass_failure = """CREATE TABLE IF NOT EXISTS hive.default.build_pass_failure (
        timestamp timestamp,
        tab varchar,
        grid varchar,
        test varchar,
        build_failure boolean,
        build_passing boolean,
        build_status varchar
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/build_pass_failure',
   format = 'PARQUET'
)"""

In [27]:
cur.execute(build_pass_failure)
cur.fetchall()

[[True]]

In [28]:
## Check if the table is there
cur.execute("select * from hive.default.build_pass_failure LIMIT 1")
cur.fetchall()

[['2021-09-20 00:00:47.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  True,
  False,
  'Fail']]

In [29]:
time_to_test = """CREATE TABLE IF NOT EXISTS hive.default.time_to_test (
        timestamp timestamp,
        tab varchar,
        grid varchar,
        test varchar,
        test_duration double
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/ai4ci/testgrid/metrics/time_to_test',
   format = 'PARQUET'
)"""

In [30]:
cur.execute(time_to_test)
cur.fetchall()

[[True]]

In [31]:
## Check if the table is there
cur.execute("select * from hive.default.time_to_test LIMIT 2")
cur.fetchall()

[[None,
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  77.64062500000001],
 [None,
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'TestInstall_test_install.cluster[openshift_version=4.6]',
  0.0010846141025641026]]

In [32]:
# TODO: location of metric file needs to be changed once smaug cluster is available
# probability_to_fail is ebing excluded from OSS demo due to resource limitations
probability_to_fail = """CREATE TABLE IF NOT EXISTS hive.default.probability_to_fail (
        timestamp timestamp,
        tab varchar,
        grid varchar,
        test varchar,
        prob double
)
WITH (
   external_location = 's3a://opf-datacatalog-zero-backup/s3_path/probability_to_fail',
   format = 'PARQUET'
)
"""

In [33]:
cur.execute(probability_to_fail)
cur.fetchall()

[[True]]

In [34]:
## Check if the table is there
cur.execute("select * from hive.default.probability_to_fail LIMIT 2")
cur.fetchall()

[['2021-09-01 16:48:28.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  0.5229258398277365],
 ['2021-08-31 15:00:03.000',
  '"redhat-assisted-installer"',
  'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
  'Overall',
  0.08448902406603673]]

## Conclusion

In this notebook, we succesfully created sql tables from metric parquet files which is a result of metric calculation on testgrid data